In [ ]:
!pip install seaborn --upgrade

# CLUE : Clustering for Mining Web URL

We aim at finding a method for digging inside a data set of URLs. The main purpose is the obtention of a clustering workflow that would allow us to directly get segments of similar URLs from stream-like data with no need for manual rule creation.

Part of this work began in the notebook APD_Structure-URL https://colab.research.google.com/drive/1MFrvUJTGmmvTe_WAI8HmI-QeTkzwXEix with some exploratory data analysis and application of URL transformation based on pattern tree construction rules.

Work presented on the current notebook is based on work from Morichetta et al.  (2017) who developed a non-specific tool for analysing massive URL data sets using text-mining methods https://www.researchgate.net/publication/312183704_CLUE_Clustering_for_Mining_Web_URLs

## Import URL data

In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn.cluster import dbscan
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize': (8,8)})

df_url = pd.read_csv('URL_dataset.csv', encoding='utf8')
df_url = df_url.loc[pd.notna(df_url['segment_name'])].drop('Unnamed: 0', axis=1)

In [ ]:
print('Total URL: %i' %len(df_url['url']))
print('Unique URL: %i' %df_url['url'].nunique())

## 1. URL-distances

URL distance computation does not have real-time constriants and can be schedule when the data collection is complete, or also on demand.

The concept of **distance** refers to a specific class of dissimilarity measures that aim at quantifying how much two points $x_1$ and $x_2$ are far apart in a numerical way. A dissimilarity measure must meet 3 propreties in order to be called a distance :

- **positivity** : $\forall (x_1, x_2), d(x_1, x_2) \geq 0$ ; $d(x_1, x_2) = 0 \iff x_1 = x_2$
- **symmetry** : $\forall (x_1, x_2), d(x_1, x_2) = d(x_2, x_1)$
- **triangle inequality** : $\forall (x_1, x_2, x_3), d(x_1, x_3) \leq d(x_1, x_2) + d(x_2, x_3)$

Let's bear in mind that in order to facilitate the clustering of URLs (which is our main goal), we want to keep distances between URLs concentrated within given ranges.

### 1.1 Levenshtein distance

The Levenshtein distance $d_{LVS}(s_1, s_2)$ assigns a unitary cost for all editing operations (insert, remove, replace). It computes an absolute distance  between pairs of strings that is **at most equal to the length of the longest string**. This has to be kept in mind as it might make the Levensthein distance inconvenient when it comes to comparing a very short URL against a very long one.

The Levenshtein distance is defined as:

$d_{LVS}(i,j) = \max(i,j)$ if $\min(i,j) = 0$

$d_{LVS}(i,j) = \min(d_{LVS}(i-1, j),~d_{LVS}(i, j-1),~d_{LVS}(i-1, j-1)+I(s_{1i}\neq s_{2i}))$ otherwise

Where $d_{LVS}(i,j)$ is the distance between the first $i$ characters of $s_1$ and the first $j$ characters of $s_2$ and $I$ is the indicator function, here equal to $0$ when $s_{1i} = s_{2j}$

In [ ]:
def levenshtein_distance(s1, s2):
    l1, l2 = len(s1), len(s2)

    if s1 == s2:
        return 0
    elif l1 == 0:
        return l2
    elif l2 == 0:
        return l1
  
    v0 = [None]*(l2+1)
    v1 = [None]*(l2+1)
    for i in range(len(v0)):
        v0[i] = i
    for i in range(l1):
        v1[0] = i+1
        for j in range(l2):
            if s1[i] == s2[j]:
                C = 0
            else:
                C = 1
            v1[j+1] = min(v1[j]+1, v0[j+1]+1, v0[j]+C)
        for j in range(len(v0)):
            v0[j] = v1[j]
        
    return v1[l2]

### 1.2 Jaro distance

The Jaro distance $d_{JRO}(s_1, s_2)$ considers **the number and the order  of common characters between two strings**.

$d_{JRO} = 1$ if $m = 0$

$d_{JRO} = 1 - \frac{1}{3}(\frac{m}{|s_1|}+\frac{m}{|s_2|}+\frac{m-t}{m})$ otherwise

where $m$ is the number of matching characters and $t$ is half the number of  transpositions.

In [ ]:
def jaro_distance(s1, s2):
    l1, l2 = len(s1), len(s2)
    
    if l1 == 0 and l2 == 0:
        return 1.
    
    match_distance = max(l1, l2)//2 - 1
    s1_matches = [None]*l1
    s2_matches = [None]*l2
    
    m, t = 0., 0.
    
    for i in range(l1):
        start = max(0, i-match_distance)
        end = min(i+match_distance+1, l2)
        
        for j in range(start, end):
            if s2_matches[j]:
                continue
            if s1[i] != s2[j]:
                continue
            s1_matches[i] = True
            s2_matches[j] = True
            m += 1.
            break
    
    if m == 0:
        return 1.
    
    k = 0
    for i in range(l1):
        if not s1_matches[k]:
            continue
        while not s2_matches[k]:
            k += 1
        if s1[i] != s2[k]:
            t += 1.
        k += 1
    
    return 1.-(1./3)*(m/l1+m/l2+(m-t/2.)/m)

### 1.3 URL distance

This distance calculation $d_{URL}$ is inspired from a custom modification of the Levenshtein distance $d_{LVS_2}$. Here total number of insertions and deletions are counted and **replacement is weighted by a factor of 2**. In the end, a replacement is considered as the combination of an insertion and a deletion. **String length is also considered and results are normalised in $[0,1]$**.

$d_{URL}(s_1, s_2) = 1 - \frac{|s_1|+|s_2|-d_{LVS_2}(s_1, s_2)}{|s_1|+|s_2|}$

we get $d_{URL}(s_1, s_2) = 0 \iff s_1 = s_2$ and $d_{URL}(s_1, s_2) = 1$ if $s_1$ and $s_2$ are totally different.

In [ ]:
def levenshtein_distance2(s1, s2):
    l1, l2 = len(s1), len(s2)

    if s1 == s2:
        return 0
    elif l1 == 0:
        return l2
    elif l2 == 0:
        return l1
  
    v0 = [None]*(l2+1)
    v1 = [None]*(l2+1)
    for i in range(len(v0)):
        v0[i] = i
    for i in range(l1):
        v1[0] = i+1
        for j in range(l2):
            if s1[i] == s2[j]:
                C = 0.
            else:
                C = 2.
            v1[j+1] = min(v1[j]+1, v0[j+1]+1, v0[j]+C)
        for j in range(len(v0)):
            v0[j] = v1[j]
        
    return v1[l2]

def url_distance(s1, s2):
    l1, l2 = len(s1), len(s2)
    
    if s1 == s2:
        return 0
    elif l1 == 0:
        return l2
    elif l2 == 0:
        return l1
    else:
        return 1. - (l1+l2-levenshtein_distance2(s1, s2))/(l1+l2)

In [ ]:
#CLUE article test example
s1, s2 = 'google.com', '1goggle.com'
print('dLVS(\'{}\', \'{}\') = %s'.format(s1, s2)  %np.round(levenshtein_distance(s1, s2),3))
print('dJRO(\'{}\', \'{}\') = %s'.format(s1, s2)  %np.round(jaro_distance(s1, s2),3))
print('dURL(\'{}\', \'{}\') = %s'.format(s1, s2)  %np.round(url_distance(s1, s2),3))

#
#dLVS('google.com', '1goggle.com') = 2
#dJRO('google.com', '1goggle.com') = 0.094
#dURL('google.com', '1goggle.com') = 0.143
#

### 1.4 Distances CDF and distance measure selection

#### 1.4.1 Format URLs 

In [ ]:
U = df_url['url'].drop_duplicates().sample(frac=0.005).tolist()
print('Unique URL: %i' %len(U))

#### 1.4.2 Levenshtein distance

In [ ]:
%%time
#calculate distances between each pair of URLs
lvs_matrix = np.zeros((len(U),len(U)), dtype=np.float)
for i in range(0, len(U)):
    for j in range(0, len(U)):
        lvs_matrix[i,j] = np.round(levenshtein_distance(U[i],U[j]),2)

#build CDF array
to_plot_lvs = lvs_matrix.ravel()
to_plot_lvs_sorted = np.sort(to_plot_lvs)
p = 1.*np.arange(len(to_plot_lvs))/(len(to_plot_lvs)-1)

#plot CDF
plot = sns.lineplot(to_plot_lvs_sorted, p)
plot.set(title='Cumulative distribution function of Levenshtein distance on the URL data set',
         xlabel='dLVS',
         ylabel='CDF');
plot.set(ylim=[0, 1]);    

#### 1.4.3 Jaro distance

In [ ]:
%%time
#calculate distances between each pair of URLs
jro_matrix = np.zeros((len(U),len(U)), dtype=np.float)
for i in range(0, len(U)):
    for j in range(0, len(U)):
        jro_matrix[i,j] = np.round(jaro_distance(U[i],U[j]),3)

#build CDF array
to_plot_jro = jro_matrix.ravel()
to_plot_jro_sorted = np.sort(to_plot_jro)
p = 1.*np.arange(len(to_plot_jro))/(len(to_plot_jro)-1)

#plot CDF
plot = sns.lineplot(to_plot_jro_sorted, p)
plot.set(title='Cumulative distribution function of Jaro distance on the URL data set',
         xlabel='dJRO',
         ylabel='CDF');
plot.set(ylim=[0, 1]);       

#### 1.4.4 URL distance

In [ ]:
%%time
#calculate distances between each pair of URLs
url_matrix = np.zeros((len(U),len(U)), dtype=np.float)
for i in range(0, len(U)):
    for j in range(0, len(U)):
        url_matrix[i,j] = np.round(url_distance(U[i],U[j]),3)
#print(url_matrix)

#build CDF array
to_plot_url = url_matrix.ravel()
to_plot_url_sorted = np.sort(to_plot_url)
p = 1.*np.arange(len(to_plot_url))/(len(to_plot_url)-1)

#plot CDF
plot = sns.lineplot(to_plot_url_sorted, p)
plot.set(title='Cumulative distribution function of URL distance on the URL data set',
         xlabel='dURL',
         ylabel='CDF');
plot.set(ylim=[0, 1]);

## 2. URL clustering

Here we aim at grouping together points according to the $d_{URL}$ distance function introduced above. URLs grouped in the same cluster share common features and we'll see if we can further consider that they represent a segment.

Clustering is performed using **DBSCAN**, a density-based clustering method, which shows several advantages over centroid-based approaches (such as *k-means clustering*):
- there is no need for prior knowledge of the total number of clusters
- arbitrarily-shaped clusters can be discovered
- outliers are considered and some may be left apart clustering as noise

Illustration : let us consider a set of points in a sample space to be clustered. Let $d(x_1, x_2)$ be the distance between two points $x_1$ and $x_2$. Consider now the sphere of radius $r$ centered in $x_1$. If at least $minPoints$ are within distance $d \leq r$ from $x_1$, then $x_1$ is classified as **core point**. The $minPoints$ are defiend as *directly reachable* from $x_1$. 
A given point $x_k$ is said to be *reachable* from $x_1$ if there exists a path $x_1, x_2, \dots, x_k$ so that $x_{i+1}$ is directly reachable from $x_i$. The subset of reachable points from $x_1$ form a **cluster**, i.e. a **dense region**. 
Points that are not reachable from $x_1$ are **outliers**, they may form a separate cluster if they fall in a separate dense region, or remain in the noise region as outliers.

Although $minPoints$ and $r$ can be tuned by the user, the critical parameter is $r$, the radius of the sphere that indeed defines the enveloppe of the cluster. A too small value of $r$ would lead to a too high number of clusters and outliers, whereas a too large value of $r$ would lead to a too small number of clusters, showing heterogeneity. Authors even warn about the fact that choice of $r$ needs a preliminary study since it can completely change clustering results.

### 2.1 Radius $r$ sensitivity analysis

We use $d_{URL}$ as the distance measure between the URLs we want to cluster. Since $d_{URL} \in [0, 1]$ by construction, we choose to make $r$ vary within a range at first. Following we compute a sort of *grid search* over parameters $r$ and $minPoints$

In [ ]:
%%time
U = df_url['url'].drop_duplicates().sample(frac=0.005).tolist()
print('Unique URL: %i\n' %len(U))
array_U = np.arange(len(U)).reshape(-1, 1)

#calculate distances between each pair of URLs using url_distance
url_matrix = np.zeros((len(U),len(U)), dtype=np.float)
for i in range(0, len(U)):
    for j in range(0, len(U)):
        url_matrix[i,j] = np.round(url_distance(U[i],U[j]),4)

clustering_results = {'r': [],
                      'N_clusters': [],
                      'Noise_elements': []}
for r in [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.75]:
    res_tmp = dbscan(url_matrix, metric='precomputed', eps=r, min_samples=8)
    n_clusters_tmp = len(np.unique(res_tmp[1]))
    n_noise_tmp = np.count_nonzero(res_tmp[1]==-1)
    clustering_results['r'].append(r)
    clustering_results['N_clusters'].append(n_clusters_tmp)
    clustering_results['Noise_elements'].append(n_noise_tmp)

In [ ]:
# clust = OPTICS(min_samples=2, xi=.05, min_cluster_size=.05)
# X = np.arange(len(U)).reshape(-1, 1)
# clust.fit(X)

# labels_a = cluster_optics_dbscan(reachability=clust.reachability_,
#                                    core_distances=clust.core_distances_,
#                                    ordering=clust.ordering_, eps=0.0005)

# labels_b = cluster_optics_dbscan(reachability=clust.reachability_,
#                                    core_distances=clust.core_distances_,
#                                    ordering=clust.ordering_, eps=0.005)

# space = np.arange(len(X))
# reachability = clust.reachability_[clust.ordering_]
# labels = clust.labels_[clust.ordering_]